In [33]:
import numpy as np

# Create monkey patches
np.float = float
np.int = int
np.object = object
np.bool = bool

In [53]:
import librosa.display
import utility_functions as utils
import importlib
from scipy.spatial.distance import jensenshannon
import pandas as pd
import numpy as np
import madmom
# Import the necessary processors
from madmom.features.tempo import TempoEstimationProcessor
from madmom.features.beats import RNNBeatProcessor
from madmom.processors import IOProcessor
from madmom.features.beats import RNNBeatProcessor, DBNBeatTrackingProcessor
from madmom.processors import SequentialProcessor
import matplotlib.pylab as plt
from glob import glob
import librosa
from scipy.stats import ttest_ind
import librosa.display
import seaborn as sns
import pandas as pd
import utility_functions as utils
import importlib
from scipy.stats import norm
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
from IPython.display import Audio

importlib.reload(utils)

data = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/test.json'

In [35]:
data_audio = utils.get_all_artist_audio(data)

In [36]:
# Here I am just loading the saved progress because calculating everything would take like 2 days but above you can see the utility functions I used to calculate the features.
df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Code/Audio/output_w_tempos_and_local_tempos.xlsx', engine='openpyxl')

In [43]:
df.rename(columns={'FilePath': 'Path', 'FileName': 'Song', 'Release_Year': 'Release Year'}, inplace=True)
df['Release Year'] = df['Release Year'].astype(int)

In [44]:
df

,Path,Song,Duration (s),Sample Rate (Hz),Coast,Tempo1,Tempo2,Energy,Spectral Centroid,Spectral Rolloff,Spectral Bandwidth,Release Year,Mean Energy,Artist,Beats,Local Tempos
0,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,8 Iz Enuff.mp3,298.840000,48000,east_coast,96.774194,48.000000,[2.83611564e-12 8.44120896e-06 3.05945038e-02 ...,[ 0. 5720.41652667 4653.92652369 ......,[ 0. 9463.84277344 8225.68359375 ......,[ 0. 3127.49309623 3015.70241681 ......,1995,137.862976,Big L,[1.5000e-01 7.7000e-01 1.4200e+00 2.0800e+00 2...,[ 96.77419355 92.30769231 90.90909091 95.23...
1,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Da Graveyard.mp3,323.760000,48000,east_coast,93.750000,46.511628,[4.19618698e-16 2.46109548e-05 1.51773983e+00 ...,[ 0. 5581.59937729 5299.67443541 ......,[ 0. 8871.6796875 8990.11230469 ......,[ 0. 3101.9549724 3021.8278237 ......,1995,116.031082,Big L,[1.2000e-01 7.7000e-01 1.4200e+00 2.0700e+00 2...,[ 92.30769231 92.30769231 92.30769231 95.23...
2,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,I Don't Understand It.mp3,260.226667,48000,east_coast,93.750000,47.244094,[1.95996759e-07 1.04421464e-05 6.28235600e+00 ...,[4201.80888552 4710.90931902 5145.42186013 ......,[8494.84863281 8828.61328125 8947.04589844 ......,[3351.40752362 3297.55375379 3166.14191736 ......,1995,98.776779,Big L,[1.3000e-01 7.7000e-01 1.4000e+00 2.0000e+00 2...,[ 93.75 95.23809524 100. 93.75...
3,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"No Endz, No Skinz.mp3",208.733333,48000,east_coast,100.000000,50.420168,[1.10115048e-13 1.51645769e-06 1.48360256e-05 ...,[ 0. 6512.52478217 4965.18006511 ......,[ 0. 9829.90722656 8397.94921875 ......,[ 0. 3211.25127137 3290.84736116 ......,1995,73.517921,Big L,[1.5000e-01 7.5000e-01 1.3800e+00 1.9700e+00 2...,[100. 95.23809524 101.69491525 96.77...
4,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,MVP.mp3,218.866667,48000,east_coast,86.956522,43.478261,[1.13744059e-14 1.56197807e-05 4.88470117e-01 ...,[ 0. 5282.25973485 4470.70845932 ......,[ 0. 8537.91503906 8171.85058594 ......,[ 0. 3037.57586169 3185.42508717 ......,1995,76.011833,Big L,[ 0.57 1.26 1.95 2.64 3.34 4.03 4...,[86.95652174 86.95652174 86.95652174 85.714285...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Stranded On Death Row (feat. Bushwic...,287.335333,48000,west_coast,90.909091,45.801527,[5.50258228e-05 7.65438136e-05 8.96513493e-05 ...,[5256.72368159 5186.02306958 2075.71661933 ......,[9011.64550781 8850.14648438 5749.36523438 ......,[3081.17114124 3103.73938201 2874.89764785 ......,1992,50.338806,Dr.Dre,[ 0.3 1.02 1.79 2.44 3.09 3.75 4...,[83.33333333 77.92207792 92.30769231 92.307692...
1045,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,238.677917,48000,west_coast,95.238095,47.244094,[1.46632022e+01 2.31116456e+01 2.80170195e+01 ...,[5156.91532791 4546.06915737 2341.52560784 ......,[9571.50878906 8204.15039062 5361.76757812 ......,[3265.29656454 3080.35198304 2505.12598391 ......,1992,69.311768,Dr.Dre,[2.2000e-01 8.6000e-01 1.5100e+00 2.1500e+00 2...,[93.75 92.30769231 93.75 95.238095...
1046,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The $20 Sack Pyramid.mp3,173.302146,48000,west_coast,88.235294,44.444444,[9.04275746e-01 1.35645008e+00 1.45811050e+00 ...,[4849.00084212 2765.61969196 3362.34043196 ......,[8828.61328125 7192.08984375 5824.73144531 ......,[3143.54023249 3093.20079096 2333.2456014 ......,1992,15.749063,Dr.Dre,[7.0000e-02 6.5000e-01 1.2400e+00 1.8000e+00 2...,[103.44827586 101.69491525 107.14285714 100. ...
1047,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The Doctor's Office (Skit) (feat. Je...,64.005813,48000,west_coast,153.846154,75.949367,[9.69122675e-05 1.65903563e-04 2.26628188e-04 ...,[5918.01709252 5863.26473045 3110.05546721 ......,[9463.84277344 9248.51074219 7816.55273438 ......,[3130.557495 3082.55156403 3306.97612112 ...

In [45]:
data_audio['Release Year'] = data_audio['Release Year'].astype(int)

In [46]:
data_audio

,Artist,Album,Coast,Path,Song,Duration (s),Sample Rate (Hz),Release Year
0,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,8 Iz Enuff.mp3,298.840000,48000,1995
1,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Da Graveyard.mp3,323.760000,48000,1995
2,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,I Don't Understand It.mp3,260.226667,48000,1995
3,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"No Endz, No Skinz.mp3",208.733333,48000,1995
4,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,MVP.mp3,218.866667,48000,1995
...,...,...,...,...,...,...,...,...
1529,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Stranded On Death Row (feat. Bushwic...,287.335333,48000,1992
1530,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,238.677917,48000,1992
1531,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The $20 Sack Pyramid.mp3,173.302146,48000,1992
1532,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The Doctor's Office (Skit) (feat. Je...,64.005813,48000,1992


In [92]:
result = pd.merge(df, data_audio, on=['Path', 'Song'], how='right')

In [93]:
result = result[['Path', 'Song', 'Tempo1', 'Tempo2']]

In [96]:
def process_audio(audio_file):
    # Processing pipeline: from raw audio to beat activations to tempo estimation
    pipeline = SequentialProcessor([
        RNNBeatProcessor(),
        TempoEstimationProcessor(fps=100)
    ])
    tempo_histogram = pipeline(audio_file)
    sorted_tempo_histogram = tempo_histogram[tempo_histogram[:, 1].argsort()[::-1]]
    top_two_tempos = sorted_tempo_histogram[:2, :]
    return top_two_tempos

In [97]:
from tqdm import tqdm  # Import tqdm for the progress bar functionality

# Assuming 'result' is your DataFrame
# Filtering the DataFrame to find rows where both 'Tempo1' and 'Tempo2' are NaN
nan_tempo_rows = result[result['Tempo1'].isna()]

# Initialize a tqdm progress bar over the DataFrame iterrows()
for index, row in tqdm(nan_tempo_rows.iterrows(), total=nan_tempo_rows.shape[0], desc='Processing Audio'):
    top_two_tempos = process_audio(row['Path'])  # Call your audio processing function
    # Update the DataFrame with new tempo values, assuming top_two_tempos returns an array with tempos
    result.at[index, 'Tempo1'] = top_two_tempos[0][0]
    result.at[index, 'Tempo2'] = top_two_tempos[0][1]

Processing Audio: 100%|██████████| 487/487 [4:19:23<00:00, 31.96s/it]    


In [99]:
result

,Path,Song,Tempo1,Tempo2
0,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,8 Iz Enuff.mp3,96.774194,48.000000
1,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Da Graveyard.mp3,93.750000,46.511628
2,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,I Don't Understand It.mp3,93.750000,47.244094
3,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"No Endz, No Skinz.mp3",100.000000,50.420168
4,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,MVP.mp3,86.956522,43.478261
...,...,...,...,...
1529,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Stranded On Death Row (feat. Bushwic...,90.909091,45.801527
1530,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,95.238095,47.244094
1531,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The $20 Sack Pyramid.mp3,88.235294,44.444444
1532,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The Doctor's Office (Skit) (feat. Je...,153.846154,75.949367


In [156]:
data_audio = utils.get_all_artist_audio(data)

In [157]:
data_audio

,Artist,Album,Coast,Path,Song,Duration (s),Sample Rate (Hz),Release Year
0,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,8 Iz Enuff.mp3,298.840000,48000,1995
1,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Da Graveyard.mp3,323.760000,48000,1995
2,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,I Don't Understand It.mp3,260.226667,48000,1995
3,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"No Endz, No Skinz.mp3",208.733333,48000,1995
4,Big L,Lifestylez Ov Da Poor and Dangerous,east_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,MVP.mp3,218.866667,48000,1995
...,...,...,...,...,...,...,...,...
1363,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - The Day the Niggaz Took Over (feat. ...,273.206000,48000,1992
1364,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"Dr. Dre - Bitches Ain't Shit (feat. Jewell, Sn...",287.207625,48000,1992
1365,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Stranded On Death Row (feat. Bushwic...,287.335333,48000,1992
1366,Dr.Dre,The Chronic,west_coast,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,238.677917,48000,1992


In [158]:
result_final = pd.merge(result, data_audio, on=['Path', 'Song'], how='right')

In [159]:
result_final = result_final[['Artist', 'Album', 'Song', 'Coast', 'Release Year', 'Tempo1', 'Tempo2', 'Duration (s)', 'Sample Rate (Hz)', 'Path']]

In [160]:
result_final

,Artist,Album,Song,Coast,Release Year,Tempo1,Tempo2,Duration (s),Sample Rate (Hz),Path
0,Big L,Lifestylez Ov Da Poor and Dangerous,8 Iz Enuff.mp3,east_coast,1995,96.774194,48.000000,298.840000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
1,Big L,Lifestylez Ov Da Poor and Dangerous,Da Graveyard.mp3,east_coast,1995,93.750000,46.511628,323.760000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
2,Big L,Lifestylez Ov Da Poor and Dangerous,I Don't Understand It.mp3,east_coast,1995,93.750000,47.244094,260.226667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
3,Big L,Lifestylez Ov Da Poor and Dangerous,"No Endz, No Skinz.mp3",east_coast,1995,100.000000,50.420168,208.733333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
4,Big L,Lifestylez Ov Da Poor and Dangerous,MVP.mp3,east_coast,1995,86.956522,43.478261,218.866667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
...,...,...,...,...,...,...,...,...,...,...
1363,Dr.Dre,The Chronic,Dr. Dre - The Day the Niggaz Took Over (feat. ...,west_coast,1992,93.750000,46.875000,273.206000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
1364,Dr.Dre,The Chronic,"Dr. Dre - Bitches Ain't Shit (feat. Jewell, Sn...",west_coast,1992,92.307692,46.153846,287.207625,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
1365,Dr.Dre,The Chronic,Dr. Dre - Stranded On Death Row (feat. Bushwic...,west_coast,1992,90.909091,45.801527,287.335333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...
1366,Dr.Dre,The Chronic,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,west_coast,1992,95.238095,47.244094,238.677917,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...


In [161]:
lyricses = []
for index, item in result_final.iterrows():
    lyrics_path = item['Path'].replace('mp3', 'txt')
    try:
        with open(lyrics_path, 'r', encoding='utf-8') as f:
            content = f.read()
            lyricses.append(content)
    except FileNotFoundError:
        print(lyrics_path)

In [163]:
result_final['Lyrics'] = lyricses

In [164]:
result_final

,Artist,Album,Song,Coast,Release Year,Tempo1,Tempo2,Duration (s),Sample Rate (Hz),Path,Lyrics
0,Big L,Lifestylez Ov Da Poor and Dangerous,8 Iz Enuff.mp3,east_coast,1995,96.774194,48.000000,298.840000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\nYo\nMy crew is in the house\nTerra, Herb M..."
1,Big L,Lifestylez Ov Da Poor and Dangerous,Da Graveyard.mp3,east_coast,1995,93.750000,46.511628,323.760000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Intro:]\nIt's the number one crew in the ...
2,Big L,Lifestylez Ov Da Poor and Dangerous,I Don't Understand It.mp3,east_coast,1995,93.750000,47.244094,260.226667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Verse 1]\nThere are too many MC's who are...
3,Big L,Lifestylez Ov Da Poor and Dangerous,"No Endz, No Skinz.mp3",east_coast,1995,100.000000,50.420168,208.733333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Verse 1:]\nLet me get to the point real q...
4,Big L,Lifestylez Ov Da Poor and Dangerous,MVP.mp3,east_coast,1995,86.956522,43.478261,218.866667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Big L]\n A yo spark up the phillies and p...
...,...,...,...,...,...,...,...,...,...,...,...
1363,Dr.Dre,The Chronic,Dr. Dre - The Day the Niggaz Took Over (feat. ...,west_coast,1992,93.750000,46.875000,273.206000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\nI'ma say this and I'ma end mine\nIf you ai...
1364,Dr.Dre,The Chronic,"Dr. Dre - Bitches Ain't Shit (feat. Jewell, Sn...",west_coast,1992,92.307692,46.153846,287.207625,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\nBitches ain't shit but hoes and tricks\n\n...
1365,Dr.Dre,The Chronic,Dr. Dre - Stranded On Death Row (feat. Bushwic...,west_coast,1992,90.909091,45.801527,287.335333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\n""Yes, it is I,"" says me\nAnd all who agree..."
1366,Dr.Dre,The Chronic,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,west_coast,1992,95.238095,47.244094,238.677917,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\nOne, two, three and to the four\nSnoop Dog..."


In [165]:
result_final[result_final['Tempo1'].isna()]

,Artist,Album,Song,Coast,Release Year,Tempo1,Tempo2,Duration (s),Sample Rate (Hz),Path,Lyrics
442,Public Enemy,Yo Bum Rush the Show,MPE.mp3,east_coast,1987,NaN,NaN,224.333333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\nPublic Enemy\n\nI'm cold gettin' busy whil...


In [166]:
a = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/east_coast/Public Enemy/Yo Bum Rush the Show/MPE.mp3'

In [167]:
t1, t2 = process_audio(a)

In [169]:
result_final.loc[442, 'Tempo1'] = t1[0]

In [170]:
result_final.loc[442, 'Tempo2'] = t2[0]

In [171]:
result_final

,Artist,Album,Song,Coast,Release Year,Tempo1,Tempo2,Duration (s),Sample Rate (Hz),Path,Lyrics
0,Big L,Lifestylez Ov Da Poor and Dangerous,8 Iz Enuff.mp3,east_coast,1995,96.774194,48.000000,298.840000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\nYo\nMy crew is in the house\nTerra, Herb M..."
1,Big L,Lifestylez Ov Da Poor and Dangerous,Da Graveyard.mp3,east_coast,1995,93.750000,46.511628,323.760000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Intro:]\nIt's the number one crew in the ...
2,Big L,Lifestylez Ov Da Poor and Dangerous,I Don't Understand It.mp3,east_coast,1995,93.750000,47.244094,260.226667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Verse 1]\nThere are too many MC's who are...
3,Big L,Lifestylez Ov Da Poor and Dangerous,"No Endz, No Skinz.mp3",east_coast,1995,100.000000,50.420168,208.733333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Verse 1:]\nLet me get to the point real q...
4,Big L,Lifestylez Ov Da Poor and Dangerous,MVP.mp3,east_coast,1995,86.956522,43.478261,218.866667,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\n[Big L]\n A yo spark up the phillies and p...
...,...,...,...,...,...,...,...,...,...,...,...
1363,Dr.Dre,The Chronic,Dr. Dre - The Day the Niggaz Took Over (feat. ...,west_coast,1992,93.750000,46.875000,273.206000,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\nI'ma say this and I'ma end mine\nIf you ai...
1364,Dr.Dre,The Chronic,"Dr. Dre - Bitches Ain't Shit (feat. Jewell, Sn...",west_coast,1992,92.307692,46.153846,287.207625,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,\n\nBitches ain't shit but hoes and tricks\n\n...
1365,Dr.Dre,The Chronic,Dr. Dre - Stranded On Death Row (feat. Bushwic...,west_coast,1992,90.909091,45.801527,287.335333,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\n""Yes, it is I,"" says me\nAnd all who agree..."
1366,Dr.Dre,The Chronic,Dr. Dre - Nuthin' but a ＂G＂ Thang (feat. Snoop...,west_coast,1992,95.238095,47.244094,238.677917,48000,/Users/borosabel/Documents/Uni/Thesis/PopMIR/D...,"\n\nOne, two, three and to the four\nSnoop Dog..."


In [172]:
result_final.to_excel("baseline_data.xlsx", sheet_name="Sheet1", index=False)

In [173]:
result_final['Sample Rate (Hz)'].unique()

array([48000, 44100])